In [2]:
import cirq
import math
import sympy
import numpy as np
from tqdm import tqdm
from scipy import optimize
import tensorflow as tf

In [ ]:
%run ./Classifiers/Rescaling.ipynb

# Gate definition

In [11]:
class iHalfSwap(cirq.TwoQubitGate):
    def __init__(self):
        super(iHalfSwap, self)
    def _unitary_(self):
        return np.array([
            [np.sqrt(2), 0.0,  0.0,     0.0    ],
            [   0.0,     1.0,  1.0j,    0.0    ],
            [   0.0,     1.0j, 1.0,     0.0    ],
            [   0.0,     0.0,  0.0,  np.sqrt(2)]
        ]) / np.sqrt(2)
    def _circuit_diagram_info_(self, args):
        return "i1/2swap", "i1/2swap"

class invIHalfSwap(cirq.TwoQubitGate):
    def __init__(self):
        super(invIHalfSwap, self)
    def _unitary_(self):
        return np.array(
            [[1.0 , 0.0, 0.0, 0.0],
            [0.0, 1/np.sqrt(2),-1/np.sqrt(2)*1j, 0.0],
            [0.0, -1/np.sqrt(2)*1j,1/np.sqrt(2), 0.0],
            [0.0, 0.0, 0.0, 1.0]])
    def _circuit_diagram_info_(self, args):
        return "invi1/2swap", "invi1/2swap"

class HalfSwap(cirq.TwoQubitGate):
    def __init__(self):
        super(HalfSwap, self)
    def _unitary_(self):
        return np.array([
            [np.sqrt(2),   0.0,      0.0,     0.0    ],
            [   0.0,     1.0+1.0j, 1.0-1.0j,  0.0    ],
            [   0.0,     1.0-1.0j, 1.0+1.0j,  0.0    ],
            [   0.0,       0.0,      0.0,   np.sqrt(2)]
        ]) / np.sqrt(2)
    def _circuit_diagram_info_(self, args):
        return "1/2swap", "1/2swap"

class invHalfSwap(cirq.TwoQubitGate):
    def __init__(self):
        super(invHalfSwap, self)
    def _unitary_(self):
        return np.array([
            [np.sqrt(2),   0.0,      0.0,     0.0    ],
            [   0.0,     0.5-0.5j, 0.5+0.5j,  0.0    ],
            [   0.0,     0.5+0.5j, 0.5-0.5j,  0.0    ],
            [   0.0,       0.0,      0.0,   np.sqrt(2)]
        ]) / np.sqrt(2)
    def _circuit_diagram_info_(self, args):
        return "inv1/2swap", "inv1/2swap"

# Classifier class

In [23]:
class QC_SVM_Classifier :
    
    def __init__(self):
        #Data
        self.x = None
        self.y = None
        self.n_features = 0
        self.n_point = 0
        
        #Quantum Circuit
        self.entang_gate = None
        self.rep = 0
        self.n_qubit = 0
        self.qubits = None
        self.n_symbols = 0
        self.symbols_i = None
        self.symbols_j = None
        self.circuit = None
        self.sim = cirq.Simulator()
        
        #SVM
        self.C = 0
        self.epsilon = 0
        self.alpha = None
        self.supportVectors = None
        self.matrix = None
        
    def set_training_data(self,x,y) :
        if np.max(x) != np.pi/2 or np.min(x) != -np.pi/2 :
            self.x = rescale(x, x_min = -np.pi/2., x_max = np.pi/2)
        else :
            self.x = x.copy()
            
        if np.min(y) == 0. :
            self.y = y.copy()*2 - 1
        else :
            self.y = y.copy()
        
        self.n_point = len(x)
        self.n_features = len(x[0])
    
    def set_training_params (self, C = 1,epsilon = 1e-8, entang_gate = "I1/2SWAP", repetitions = 1000):
        self.C = C
        self.epsilon = epsilon
        self.entang_gate = entang_gate
        self.rep = repetitions

    def create_circuit(self):
        self.n_qubit = math.ceil(self.n_features/6)
        self.qubits = cirq.GridQubit.rect(1,self.n_qubit)

        self.symbols_i = sympy.symbols('theta_i_0:'+str(self.n_qubit*6))
        self.symbols_j = sympy.symbols('theta_j_0:'+str(self.n_qubit*6))
        
        self.n_symbols = 6 * self.n_qubit

        circuit_i = cirq.Circuit()
        circuit_j = cirq.Circuit()

        circuit_i.append([cirq.H(q) for q in self.qubits])
        
        circuit_i.append([cirq.rx(self.symbols_i[ind])(self.qubits[ind]) for ind in range(self.n_qubit)])
        circuit_j.append([cirq.rz(-self.symbols_j[ind+5*self.n_qubit])(self.qubits[ind]) for ind in range(self.n_qubit)])
        
        circuit_i.append([cirq.ry(self.symbols_i[ind+self.n_qubit])(self.qubits[ind]) for ind in range(self.n_qubit)])
        circuit_j.append([cirq.ry(-self.symbols_j[ind+4*self.n_qubit])(self.qubits[ind]) for ind in range(self.n_qubit)])
        
        circuit_i.append([cirq.rz(self.symbols_i[ind+2*self.n_qubit])(self.qubits[ind]) for ind in range(self.n_qubit)])
        circuit_j.append([cirq.rx(-self.symbols_j[ind+3*self.n_qubit])(self.qubits[ind]) for ind in range(self.n_qubit)])
        
        gates_name = ["SWAP","ISWAP","1/2SWAP","I1/2SWAP","CNOT","CZ","XX","YY","ZZ"]
        gates = [cirq.SWAP,cirq.ISWAP,HalfSwap(),iHalfSwap(),cirq.CNOT,cirq.CZ,cirq.XX,cirq.YY,cirq.ZZ]
        
        if self.entang_gate in gates_name  :
            gate = gates[gates_name.index(self.entang_gate)]

        if self.entang_gate == "I1/2SWAP" :
            inv_gate = invIHalfSwap()
        elif self.entang_gate == "1/2SWAP" :
            inv_gate = invHalfSwap()
        else:
            inv_gate = cirq.inverse(gate)

        for ind in range(int(self.n_qubit/2)) :
            circuit_i.append(gate(self.qubits[2*ind],self.qubits[2*ind+1]))
        for ind in range(int(self.n_qubit/2)-1):
            circuit_i.append(gate(self.qubits[2*ind+1],self.qubits[2*(ind+1)]))

        for ind in range(int(self.n_qubit/2)-1) :
            circuit_j.append(inv_gate(self.qubits[2*ind+1],self.qubits[2*(ind+1)]))
        for ind in range(int(self.n_qubit/2)):
            circuit_j.append(inv_gate(self.qubits[2*ind],self.qubits[2*ind+1]))

        circuit_i.append([cirq.rx(self.symbols_i[ind+3*self.n_qubit])(self.qubits[ind]) for ind in range(self.n_qubit)])
        circuit_j.append([cirq.rz(-self.symbols_j[ind+2*self.n_qubit])(self.qubits[ind]) for ind in range(self.n_qubit)])

        circuit_i.append([cirq.ry(self.symbols_i[ind+4*self.n_qubit])(self.qubits[ind]) for ind in range(self.n_qubit)])
        circuit_j.append([cirq.ry(-self.symbols_j[ind+self.n_qubit])(self.qubits[ind]) for ind in range(self.n_qubit)])

        circuit_i.append([cirq.rz(self.symbols_i[ind+5*self.n_qubit])(self.qubits[ind]) for ind in range(self.n_qubit)])
        circuit_j.append([cirq.rx(-self.symbols_j[ind])(self.qubits[ind]) for ind in range(self.n_qubit)])

        circuit_j.append([cirq.H(q) for q in self.qubits])

        self.circuit = circuit_i + circuit_j
        self.circuit.append(cirq.measure(*[q for q in self.qubits],key = "z"))

    def set_sym_val(self,x_i,x_j):
        dico_res = {}
        for ind in range(self.n_features):
            dico_res[self.symbols_i[ind]] = x_i[ind]
            dico_res[self.symbols_j[ind]] = x_j[ind]
        for ind in range(self.n_features,self.n_symbols):
            dico_res[self.symbols_i[ind]] = 0
            dico_res[self.symbols_j[ind]] = 0
        return dico_res

    def compute_dist(self,x_1, x_2) :
        resov = cirq.ParamResolver(self.set_sym_val(x_1,x_2))
        res_sim = self.sim.run(self.circuit,resov,repetitions = self.rep)
        return res_sim.histogram(key="z")[0]/self.rep
    
    def compute_matrix(self) :
        self.create_circuit()
        self.matrix = np.ones((self.n_point,self.n_point))
        for i in tqdm(range(1,self.n_point)):
            for j in range(i):
                self.matrix[i,j] = self.compute_dist(self.x[i],self.x[j])
                self.matrix[j,i] = self.matrix[i,j]
                
    def save_matrix(self,name):
        np.save(name,self.matrix)
    
    def load_matrix(self,file_path):
        try :
            self.matrix = np.load(file_path)
        except :
            print("File not found")
    
    
    def fit(self):
        yp = self.y.reshape(-1, 1)
        GramHXy = self.matrix * np.matmul(yp, yp.T)

        # Lagrange dual problem
        def Ld0(G, alpha):
            return alpha.sum() - 0.5 * alpha.dot(alpha.dot(G))

        # Partial derivate of Ld on alpha
        def Ld0dAlpha(G, alpha):
            return np.ones_like(alpha) - alpha.dot(G)

        # Constraints on alpha of the shape :
        A = np.vstack((-np.eye(self.n_point), np.eye(self.n_point)))
        b = np.hstack((np.zeros(self.n_point), self.C * np.ones(self.n_point)))
        constraints = ({'type': 'eq',   'fun': lambda a: np.dot(a, self.y),     'jac': lambda a: self.y},
                       {'type': 'ineq', 'fun': lambda a: b - np.dot(A, a), 'jac': lambda a: -A})

        # Maximize by minimizing the opposite
        optRes = optimize.minimize(fun=lambda a: -Ld0(GramHXy, a),
                                   x0=np.ones(self.n_point), 
                                   method='SLSQP', 
                                   jac=lambda a: -Ld0dAlpha(GramHXy, a), 
                                   constraints=constraints)
        
        self.alpha = optRes.x
        
        supportIndices = self.alpha > self.epsilon
        self.supportVectors = self.x[supportIndices]
        self.supportAlphaY = self.y[supportIndices] * self.alpha[supportIndices]
        
    def training(self, x, y, C = 1, epsilon = 1e-6, entang_gate = "I1/2SWAP", repetitions = 1000) :
        print("Setting training Data and parameters")
        self.set_training_data(x,y)
        self.set_training_params (C,epsilon, entang_gate, repetitions)
        print("Computing matrix (increasing time for each iteration)")
        self.compute_matrix()
        print("Training the SVM")
        self.fit()
        print("Training completed")
    
    def predict_val(self, x):
            x1 = np.apply_along_axis(lambda s: self.compute_dist(s, x), 1, self.supportVectors)
            x2 = x1 * self.supportAlphaY
            return np.sum(x2)
    
    def predict(self, x_cl):
        """ Predict y values in {-1, 1} """        
        vals = []
        for el in tqdm(x_cl):
            vals.append(self.predict_val(el))
        return np.array([int(el > 0) for el in vals])